http://www.sports-reference.com/cbb/postseason/1939-ncaa.html

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import element
from time import sleep
import pandas as pd


def get_page_data(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

def tidy_location(locStr):
    if locStr.startswith('at '):
        return locStr[len('at '):]

In [45]:


def get_bracket_85_on():    
    years = range(2017, 1984, -1)
    #TODO fix years earlier than 1985
    bracket = []
    for year in years: 
        soup = get_page_data(year)
        region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')        
        for region_tag in region_tags:        
            region = str(region_tag.string).lower()
            if region is not 'final four':            
                search = {'id': region}        
                region_div = soup.find('div',**search)
                if region_div is not None:
                    region_first_round = region_div.div.div                
                    for elem in region_first_round.contents:
                        if isinstance(elem, element.Tag ):
                            seeds_loc = elem.find_all('span')
                            if len(seeds_loc) > 2:
                                seed1 = seeds_loc[0].string
                                seed2 = seeds_loc[1].string
                                currLoc = tidy_location(seeds_loc[2].string)        
                                alist = elem.select('div a')
                                team1 = alist[0].string
                                team2 = alist[2].string        
                                bracketRow = [year, region, seed1, team1, currLoc, seed2, team2]
                                bracket.append(bracketRow) 
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Seed', 'Team Name','Location', 'Oppenent Seed', 'Opponent']) 

df = get_bracket_85_on()
#df.to_csv('ncaa.csv')
df


,Year,Region,Seed,Team Name,Location,Oppenent Seed,Opponent
0,2017,midwest,1,Kansas,"Tulsa, OK",16,UC-Davis
1,2017,midwest,8,Miami (FL),"Tulsa, OK",9,Michigan State
2,2017,midwest,5,Iowa State,"Milwaukee, WI",12,Nevada
3,2017,midwest,4,Purdue,"Milwaukee, WI",13,Vermont
4,2017,midwest,6,Creighton,"Sacramento, CA",11,Rhode Island
5,2017,midwest,3,Oregon,"Sacramento, CA",14,Iona
6,2017,midwest,7,Michigan,"Indianapolis, IN",10,Oklahoma State
7,2017,midwest,2,Louisville,"Indianapolis, IN",15,Jacksonville State
8,2017,west,1,Gonzaga,"Salt Lake City, UT",16,South Dakota State
9,2017,west,8,Northwestern,"Salt Lake City, UT",9,Vanderbilt


In [37]:
def get_bracket_84():
    soup = get_page_data(1984)
    region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')   
    bracket = []
    for region_tag in region_tags:        
        region = str(region_tag.string).lower()
        print("region ", region)
        if region is not 'final four':            
            search = {'id': region}        
            region_div = soup.find('div',**search)
            if region_div is not None:
                #region_first_round = region_div.div.div
                bracket_rounds = region_div.div
                div_rounds = bracket_rounds.find_all('div', {'class':'round'})                
                bracket_round = 1
                for div_round in div_rounds:
                    print("round, ", bracket_round)
                    if isinstance(div_round, element.Tag ):
                        #print(len(div_round))
                        for gm in div_round:
                            if isinstance(gm, element.Tag ):
                                #game_data = gm.find_all('span')
                                print(gm)
                                print("len contents: ", len(gm.contents))
                                print()
                                if (len(gm.contents) >= 9):
                                    seeds_loc = gm.find_all('span')
                                    if len(seeds_loc) > 2:
                                        seed1 = seeds_loc[0].string
                                        seed2 = seeds_loc[1].string                                        
                                        currLoc = tidy_location(seeds_loc[2].string)        
                                        alist = gm.select('div a')
                                        team1 = alist[0].string
                                        team2 = alist[2].string
                                        bracketRow = [1984, region, bracket_round, seed1, team1, currLoc, seed2, team2]
                                        bracket.append(bracketRow) 
                                #if len(game_data) > 2:
#                                     seed1 = game_data[0].string
#                                     seed2 = game_data[1].string
#                                     currLoc = tidy_location(game_data[2].string)        
#                                     alist = div_round.select('div a')
#                                     team1 = alist[0].string
#                                     team2 = alist[2].string        
#                                     bracketRow = [1984, region, bracket_round, seed1, team1, currLoc, seed2, team2]
#                                     #print(bracketRow)
#                                     bracket.append(bracketRow) 

                        bracket_round += 1
      
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Round', 'Seed', 'Team Name','Location', 'Oppenent Seed', 'Opponent']) 


    
df84 = get_bracket_84()
df84

region  east
round,  1
<div class="bye"> <!-- game -->
<div> <!-- team -->
<span></span>
<a href="/cbb/schools/north-carolina/1984.html">UNC</a>
</div>
<div> <!-- team -->
</div>
</div>
len contents:  7

<div> <!-- game -->
<div class="winner"> <!-- team -->
<span>8</span>
<a href="/cbb/schools/temple/1984.html">Temple</a>
<a href="/cbb/boxscores/1984-03-15-st-johns-ny.html">65</a>
</div>
<div> <!-- team -->
<span>9</span>
<a href="/cbb/schools/st-johns-ny/1984.html">St. John's (NY)</a>
<a href="/cbb/boxscores/1984-03-15-st-johns-ny.html">63</a>
</div>
<span><a href="/cbb/boxscores/1984-03-15-st-johns-ny.html">at Charlotte, NC</a></span>
</div>
len contents:  9

<div> <!-- game -->
<div> <!-- team -->
<span>5</span>
<a href="/cbb/schools/auburn/1984.html">Auburn</a>
<a href="/cbb/boxscores/1984-03-15-auburn.html">71</a>
</div>
<div class="winner"> <!-- team -->
<span>12</span>
<a href="/cbb/schools/richmond/1984.html">Richmond</a>
<a href="/cbb/boxscores/1984-03-15-auburn.html">72</a>


,Year,Region,Round,Seed,Team Name,Location,Oppenent Seed,Opponent
0,1984,east,1,8,Temple,"Charlotte, NC",9,St. John's (NY)
1,1984,east,1,5,Auburn,"Charlotte, NC",12,Richmond
2,1984,east,1,6,VCU,"East Rutherford, NJ",11,Northeastern
3,1984,east,1,7,Virginia,"East Rutherford, NJ",10,Iona
4,1984,east,2,1,UNC,"Charlotte, NC",8,Temple
5,1984,east,2,12,Richmond,"Charlotte, NC",4,Indiana
6,1984,east,2,6,VCU,"East Rutherford, NJ",3,Syracuse
7,1984,east,2,7,Virginia,"East Rutherford, NJ",2,Arkansas
8,1984,east,3,1,UNC,"Atlanta, GA",4,Indiana
9,1984,east,3,3,Syracuse,"Atlanta, GA",7,Virginia


In [35]:
bracket_2017_and_2016 = df.head(64)
sample = bracket_2017_and_2016.head(8)

In [36]:
sample_group = sample.groupby(['Team Name', 'Opponent'])


In [37]:
miami = df['Team Name'].value_counts()['Miami (FL)'] + df['Opponent'].value_counts()['Miami (FL)']
uf = df['Team Name'].value_counts()['Florida'] + df['Opponent'].value_counts()['Florida']
fsu = df['Team Name'].value_counts()['Florida State'] + df['Opponent'].value_counts()['Florida State']

print("UF {} appearances, FSU {} appearances, Miami {} appearances".format(uf, fsu, miami))

UF 19 appearances, FSU 11 appearances, Miami 8 appearances
